In [68]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, RNN
import keras
import numpy as np
import pandas as pd
from scipy.sparse import csgraph
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
def labelencode_dataframe(dataframe_columns, classlist, column_names):
    le = LabelEncoder().fit(classlist)
    if len(dataframe_columns.shape) == 1:
        df_new = pd.DataFrame(le.transform(dataframe_columns), columns=column_names)
    else:
        df_new = dataframe_columns.apply(le.transform)
    return df_new

In [3]:
def dataframe_tolist(dataframe):
    ret_list = []
    headers = list(dataframe)
    for header in headers:
        ret_list.extend(list(set(dataframe[header])))
    return list(set(ret_list))

In [4]:
data = pd.read_csv('psl_data_wStats.csv', header=0, delimiter=',')

In [87]:
#Separating all features for encoding
teams = list(set(sum([list(data[['team1']]), list(data['team2'])], [])))
venues = list(set((data['venue'])))
toss_decisions = list(set(data['toss_decision']))
#encoding.
team1_new = labelencode_dataframe(data['team1'], teams, ['team1'])
team2_new = labelencode_dataframe(data['team2'], teams, ['team2'])
toss_winner_new = labelencode_dataframe(data['toss_winner'], teams, ['toss_winner'])
winner_new = labelencode_dataframe(data['winner'], teams, ['winner'])
venue_new = labelencode_dataframe(data['venue'], venues, ['venue'])
toss_decision_new = labelencode_dataframe(data['toss_decision'], toss_decisions, ['toss_decision'])

#New Encoded Data
data_new = pd.concat([team1_new, team2_new, data.iloc[:,2:12], venue_new, toss_winner_new, toss_decision_new], axis=1)
#Features
X = data_new.iloc[:,0:27]
#Labels
Y = pd.DataFrame(np.where(data['team1']==data['winner'], 1, 0))

# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [88]:
model = Sequential()

model.add(Embedding(2500, 128, input_length = 15, dropout = 0.2))
model.add(LSTM(2500, dropout_U = 0.02, dropout_W = 0.2))
model.add(Dense(1,activation='softmax'))

model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

D:\Installations\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Installations\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(2500, dropout=0.2, recurrent_dropout=0.02)`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 15, 128)           320000    
_________________________________________________________________
lstm_14 (LSTM)               (None, 2500)              26290000  
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 2501      
Total params: 26,612,501
Trainable params: 26,612,501
Non-trainable params: 0
_________________________________________________________________
None


In [89]:
model.fit(X, Y, epochs=5)

Epoch 1/5
82/82 [==============================] - 27s 327ms/step - loss: 8.9433 - acc: 0.4390
Epoch 2/5
82/82 [==============================] - 9s 111ms/step - loss: 8.9433 - acc: 0.4390
Epoch 3/5
82/82 [==============================] - 9s 112ms/step - loss: 8.9433 - acc: 0.4390
Epoch 4/5
82/82 [==============================] - 9s 112ms/step - loss: 8.9433 - acc: 0.4390
Epoch 5/5
82/82 [==============================] - 9s 115ms/step - loss: 8.9433 - acc: 0.4390


In [90]:
score, acc = model.evaluate(X, Y)

82/82 [==============================] - 4s 45ms/step


In [94]:
print('Accuracy: %.2f' % (acc * 100))

Accuracy: 43.90
